# minimal

When using negative chunking with zarr, values are not correct

In [1]:
import zarr
import numpy as np
print("np   : ", np.__version__)
print("zarr : ", zarr.__version__)

np   :  1.18.1
zarr :  2.3.2


In [2]:
fichier1 = "/tmp/test.zarr"

example = np.zeros((10))
zh = zarr.open(fichier1, "w")

var1 = zh.create_dataset("var1", 
    shape=example.shape, 
    dtype=example.dtype,      
    chunks=(10)
)
var1[:] = 2.
print("var1 : ", var1[:])  # output as expected

var2 = zh.create_dataset("var2", 
    shape=example.shape, 
    dtype=example.dtype,      
    chunks=(-1)
)
var2[:] = 7.
print("var2 : ", var2[:])  # should be 7 but is 2 as var1


var1 :  [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
var2 :  [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


## more funny

when running the same code twice, the corrupted memory don't have the same values

In [3]:
def create():
    """this function demonstrate memory problem"""
    
    fichier1 = "/tmp/test.zarr"
    
    # first example is correct
    example1 = np.zeros((10)) + 3
    zh = zarr.open(fichier1, "w")
    var1 = zh.create_dataset("var1", 
            shape=example1.shape, 
            dtype=example1.dtype,      
            chunks=(10))
    var1[:] = example1 + 5
    print("var1 : ", var1[:])

    # second example show corrupted values
    example2 = np.zeros((10))
    var2 = zh.create_dataset("var2", 
        shape=example2.shape, 
        dtype=example2.dtype,      
        chunks=(-1))
    var2[:] = 5
    print("var2 : ", var2[:])  #should output 5, but output 8 ?

    # exact same example as before, but show other corrupted value
    example2 = np.zeros((10))
    var2 = zh.create_dataset("var3", 
        shape=example2.shape, 
        dtype=example2.dtype,      
        chunks=(-1))
    var2[:] = 5
    print("var2 : ", var2[:])  # should output 5, but output 0 ?


In [4]:
create()

var1 :  [8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
var2 :  [8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
var2 :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
